# Введение в машинное обучение

## Семинар #3

### Екатерина Кондратьева

ekaterina.kondrateva@skoltech.ru

## Регуляризация в линейных моделях. Метод Ближайших Соседей (KNN)

## 1. Регуляризация в линейных моделях

Регуляризация - наложение ограничения на решающае правило через нормы. Это нужно для борьбы с переобучением методом отбора признаков. 

Источники:
1. https://github.com/esokolov/ml-course-hse/blob/master/2018-fall/lecture-notes/lecture03-linregr.pdf    
2. http://www.machinelearning.ru/wiki/images/7/7e/VetrovSem11_LARS.pdf
3. https://ru.coursera.org/lecture/supervised-learning/rieghuliarizatsiia-sR94Q
4. https://towardsdatascience.com/regularization-in-machine-learning-connecting-the-dots-c6e030bfaddd

In [ ]:
#linear algebra
import numpy as np
#data structures
import pandas as pd
#ml models
import scipy as sp
import sklearn
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.svm import SVR
#plots
import matplotlib.pyplot as plt
%matplotlib inline
#beautiful plots
import seaborn as sns
#linear regression
import statsmodels.api as sm
#set style for plots
sns.set_style('darkgrid')
#off the warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.neighbors import KNeighborsClassifier     #KNN
from sklearn.linear_model import LogisticRegression    #Logistic Regression
from sklearn.model_selection import train_test_split

### Начнем Логистической регрессии:

Посмотрим на датасет предсказания рака груди:

In [ ]:
#load the breast cancer data and few EDA
cancer = load_breast_cancer()
print(cancer.DESCR)

In [ ]:
# Разобьем выборку на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, stratify=cancer.target, random_state=42)

log_reg = LogisticRegression() 
log_reg.fit(X_train, y_train)

print('Точность на обучающей выборке: {:.3f}'.format(log_reg.score(X_train,y_train)))
print('Точность на тестовой выборке: {:.3f}'.format(log_reg.score(X_test,y_test)))

In [ ]:
log_reg.score(X_train,y_train)

In [ ]:
y_pred = log_reg.predict(X_train)

In [ ]:
accuracy_score(y_pred, y_train)

In [ ]:
cancer.data.shape, X_train.shape

#### Посмотрим на коэффициенты модели:

In [ ]:
log_reg?

In [ ]:
cancer.data.shape[1]

In [ ]:
log_reg.coef_[0]

In [ ]:
cancer.feature_names

In [ ]:
log_reg.coef_[0]

In [ ]:
# Построим график значимости признаков (веса регрессионной модели)
n_feature = cancer.data.shape[1]
plt.barh(range(n_feature), abs(log_reg.coef_[0]), align='center') # to look at the coefs
plt.yticks(np.arange(n_feature), cancer.feature_names)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.show()

In [ ]:
LogisticRegression(C= 0.1, penalty='l1')

**Какая регуляризация используется в модели?**

Наиболее полная справка о построенной модели доступна в библиотеке `statsmodels.api`, однако в ней не поддреживаются некоторые регуляризации и их нужно прописывать формульно. В `statsmodels.api` вы можете получить полный вывод для составления уравнения регрессии, с p-value и интервалом достоверности, огранизованным в виде таблицы.


Однако, чаще используется `sklearn.linear_model.LogisticRegression()`, в котором частично информацию можно получить через `.feature_importances` или `.coef`.

In [ ]:
# добавим константу, чтобы посчитать нулевой коэффициент
X = sm.add_constant(X_train)
# подгрузим модель
model = sm.OLS(y_train, X)

results = model.fit()

In [ ]:
results.summary2()

In [ ]:
results.params

In [ ]:
# Построим график значимости признаков
n_feature = cancer.data.shape[1]
plt.barh(range(n_feature), results.params[1:], align='center') # to look at the coefs
plt.yticks(np.arange(n_feature), cancer.feature_names)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.show()

# Регуляризация

## L1 Lasso regularisation:

Объяснение ближе к математическому: 
    https://stats.stackexchange.com/questions/45643/why-l1-norm-for-sparse-models

In [ ]:
log_reg = LogisticRegression(penalty='l1') # посмотрим, что это значит в справке
log_reg.fit(X_train, y_train)

print('Accuracy on the training set: {:.3f}'.format(log_reg.score(X_train,y_train)))
print('Accuracy on the test set: {:.3f}'.format(log_reg.score(X_test,y_test)))

In [ ]:
# График значимости
n_feature = cancer.data.shape[1]
plt.barh(range(n_feature), log_reg.coef_[0], align='center') # to look at the coefs
plt.yticks(np.arange(n_feature), cancer.feature_names)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.show()

In [ ]:
log_reg.coef_[log_reg.coef_ != 0]

In [ ]:
# len(log_reg.coef_[log_reg.coef_!=0])

Нужно ли стандартизовывать данные перед обучением и как это влияет на построение модели?

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)

In [ ]:
# посмотрим на краевые значения
X_train.max(), X_train.min()

In [ ]:
X_train_sc.max(), X_train_sc.min()

In [ ]:
log_reg = LogisticRegression() # check the model params
log_reg.fit(X_train, y_train)

print('Accuracy on the training set: {:.3f}'.format(log_reg.score(X_train,y_train)))
print('Accuracy on the training set: {:.3f}'.format(log_reg.score(X_test,y_test)))

#Feature Importance
n_feature = cancer.data.shape[1]
plt.barh(range(n_feature), log_reg.coef_[0], align='center') # to look at the coefs
plt.yticks(np.arange(n_feature), cancer.feature_names)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.show()

In [ ]:
log_reg = LogisticRegression(penalty='l1', C=0.001) # вторая
log_reg.fit(X_train, y_train)

print('Accuracy on the training set: {:.3f}'.format(log_reg.score(X_train,y_train)))
print('Accuracy on the training set: {:.3f}'.format(log_reg.score(X_test,y_test)))

#Feature Importance
n_feature = cancer.data.shape[1]
plt.barh(range(n_feature), log_reg.coef_[0], align='center') # to look at the coefs
plt.yticks(np.arange(n_feature), cancer.feature_names)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.show()

#### Сколько осталось ненулевых коэффициентов после применения резуляризации?

In [ ]:
len(log_reg.coef_[log_reg.coef_!=0])

####  Насколько мы можем верить тому, что на 3х признаках точность упала всего на 3%?

Что если повторить эксперимент с регуляризацией на 5-10 различных разбиениях 
`random_seed` или на кросс-валидации? (ДЗ)

## Elastic net (L1 & L2 regularisation):

Пример работы совмещенной регуляризации, на практике применим только в специфичных задачах

In [ ]:
model

In [ ]:
results = model.fit_regularized() # method='elastic_net'

In [ ]:
#Feature Importance
n_feature = cancer.data.shape[1]
plt.barh(range(n_feature), results.params[1:], align='center') # to look at the coefs
plt.yticks(np.arange(n_feature), cancer.feature_names)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.show()

In [ ]:
results.params

In [ ]:
len(results.params)

In [ ]:
plt.barh(range(n_feature), abs(results.params[1:]), align='center') # to look at the coefs
plt.yticks(np.arange(n_feature), cancer.feature_names)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.show()

### Вопросы:

- Помогла ли l1 "жесткая" регуляризация избежать переобучения?
- Переобучилась ли модель на нескольких характеристиках?
- Как это проверить?

## 2. k Nearest Neighbors



`Sklearn`:
<a href='http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html'>Regressor</a>
<a href='http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html'>Classifier</a>

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

Тренируем

In [ ]:
iris = load_iris()
X = iris.data
y = iris.target

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


#######
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

In [ ]:
clf

## 1. Метрики и ядерные функции:

### Метрики, реализованные в `sklearn`:

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html

### Специальные метрики для токенизации в пакете:
https://pypi.org/project/Distance/

### Или здесь:

http://www.nltk.org/_modules/nltk/align/bleu_score.html

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

Какая метрика стоит по дефолту?

In [ ]:
knn

Классифицируем попарно для того, чтобы сделать отрисовку решающего правила. Это наглядный пример работы классификатора

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

pair=[0, 1]
X = iris.data[:, [0, 1]]
y = iris.target

n_classes = 3
plot_colors = ['g', 'gold', 'black']
plot_step = 0.005

x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                     np.arange(y_min, y_max, plot_step))


clf = KNeighborsClassifier(n_neighbors=10).fit(X, y)

Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
cs = plt.contourf(xx, yy, Z, cmap='Accent')

plt.xlabel(iris.feature_names[pair[0]])
plt.ylabel(iris.feature_names[pair[1]])

for i, color in zip(range(n_classes), plot_colors):
    idx = np.where(y == i)
    plt.scatter(X[idx, 0], X[idx, 1], c=color, label=iris.target_names[i],
                cmap=plt.cm.Paired)

In [ ]:
clf.score(X,y)

In [ ]:
iris.data.shape

## 2.KNN Regression:

Посмотрим на пример искусственных двумерных данных (оранжевый) и предсказание этих данных по координате с помощью метода `knn`

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

rng = np.random.RandomState(1)
X = np.sort(5 * rng.rand(80, 1), axis=0)
y = np.sin(X).ravel()
# y = np.piecewise(X.flatten(), 
#                  [X.flatten() < 3, X.flatten() >= 3], [-1, 1]).ravel()
y[::2] += 1 * (0.5 - rng.rand(40))

X_test = np.arange(0.0, 5.0, 0.01)[:, np.newaxis]

# clf = KNeighborsRegressor(n_neighbors=30, 
#                           #weights=gaussian_kernel
#                          ).fit(X, y)
clf = LinearRegression().fit(X, y)

y_ = clf.predict(X_test)
plt.scatter(X, y, c='darkorange', label='data')
plt.plot(X_test, y_, c='cornflowerblue', label='prediction');

## Какое оптимальное число соседей `k`?

In [ ]:
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV

In [ ]:
X = iris.data
y = iris.target

In [ ]:
# "отбелим данные"
standardizer = StandardScaler()
X_std = standardizer.fit_transform(X)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean', n_jobs=-1).fit(X_std, y)

In [ ]:
# новый класс объектов - пайплайн
pipe = Pipeline([('standardizer', standardizer), ('knn', knn)])

# построим сетку поиска
search_space = [{'knn__n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}]

In [ ]:
# и обозначим объект класса гридсерч или поиск по сетке
clf = GridSearchCV(pipe, search_space, cv=5, verbose=0).fit(X_std, y)

In [ ]:
# выведем параметры лучшей модели `best_estimator`
clf.best_estimator_.get_params()['knn__n_neighbors']

In [ ]:
range(1,15)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, 
                                                    stratify=cancer.target, random_state=42)

training_accuracy = []
test_accuracy = []

#try KNN for diffrent k nearest neighbor from 1 to 15
neighbors_setting = range(1,15)

for n_neighbors in neighbors_setting:
#     print(n_neighbors)
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X_train,y_train)
    training_accuracy.append(knn.score(X_train, y_train))
    test_accuracy.append(knn.score(X_test, y_test))
 
plt.plot(neighbors_setting, training_accuracy, label='Accuracy of the training set')
plt.plot(neighbors_setting, test_accuracy, label='Accuracy of the test set')
plt.ylabel('Accuracy')
plt.xlabel('Number of Neighbors')
plt.legend()

#by looking at plot, best result accurs when n_neighbors is 6

In [ ]:
y_test.shape

In [ ]:
knn.score(X_test,y_test)

#### Как узнать, какие признаки были билее весомые для классификации?

В `sklearn` нет реализованого метода подсчета важности признака для этого типа классифкаторов.
Существуют методы которые позволяют косвенно оценить значимость признака - убирая его из выборки или заменяя радномными значениями. Или более сложные методы, как https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3281073/ 

Мы этим заниматься не будем, так как всегда проще интерпретировать модель Логистической регрессии (Линейной регрессии), даже если она немного проигрывает по точности.

### Самостоятельная работа:

1. Заполнить пропуски (пример кросс-валидации):

In [ ]:
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits = 10, random_state = 42)
X = cancer.data
y = cancer.target
i = 0

for train_index, test_index in kfold.split(X, y):
    #("TRAIN:", train_index, "TEST:", test_index)
    <YOUR CODE> # split train nd test
    print('Fold #', i)
    i+=1
    <YOUR CODE> # fit model
    print(knn.score(X_test,y_test))
          

2. Повторить эксперимент с регуляризацией (и без на первой выборке) на 5-10 различных разбиениях 
`random_seed` или на кросс-валидации? (ДЗ)

## Унести домой: 

- l1 и l2 нормы для регуляризации
- kNN - очень популярный алгоритм в тех случаях, когда объекты сравнимы и имеет место "похожесть" объектов.
- kNN сильно зависит от метрики. А значит и от масштаба признаков. Перед применением нужно привести признаки к одной шкале
- kNN плохо воспринимает большое (>100) количество признаков, т.к. объекты оказываются одинаково отдалены друг от друга в таких пространствах 